In [9]:
# !pip install broad_babel polar

In [10]:
import os
import requests
from pathlib import Path
import polars as pl
import pandas as pd
from broad_babel.query import get_mapper

In [11]:
# Define paths
DATA_DIR = Path("..") / "data"

# Main feature data
PARQUET_PATH = DATA_DIR / "profiles_var_mad_int.parquet"
URL = "https://cellpainting-gallery.s3.amazonaws.com/cpg0016-jump-assembled/source_all/workspace/profiles_assembled/COMPOUND/v1.0/profiles_var_mad_int.parquet"

In [12]:
# Create folder if it doesn't exist
DATA_DIR.mkdir(exist_ok=True)

# 1. Check for the main Parquet file
if not PARQUET_PATH.exists():
    print(f"File not found. Downloading from {URL}...")
    print("This is a ~2.4GB file, so this may take a few minutes...")
    
    response = requests.get(URL, stream=True)
    with open(PARQUET_PATH, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024*1024): # 1MB chunks
            if chunk:
                f.write(chunk)
    print("Download Complete!")
else:
    print(f"Found existing data at {PARQUET_PATH.resolve()}")

Found existing data at C:\Users\nikhi\OneDrive\Documents\Bonsai\CellPainting\data\profiles_var_mad_int.parquet


In [ ]:
# 1. Use Polars to scan unique plates and SORT them for consistent order
lazy_df = pl.scan_parquet(PARQUET_PATH)
unique_plates = lazy_df.select(
    pl.col("Metadata_Plate").unique().sort() 
).collect()["Metadata_Plate"].to_list()

print(f"Total Unique Plates found: {len(unique_plates)}")

# 2. SELECT PLATE BY INDEX
# Change this number to select a different plate
plate_index = 0 

selected_plate = unique_plates[plate_index]
print(f"Selected sample plate {plate_index}: {selected_plate}")

Total Unique Plates found: 1713
Selected sample plate 0: 1053597806


In [14]:
# 1. Filter and collect the data for just that plate
print(f"Processing data for {selected_plate}...")
plate_data = lazy_df.filter(pl.col("Metadata_Plate") == selected_plate).collect()

# 2. Extract unique JCP2022 IDs to query broad_babel
# (broad_babel is a query tool that translates JUMP-CP's internal IDs into human-readable drug names and control labels)
# (Babel requires a tuple, just like the tutorial mentioned)
jcp_ids = tuple(plate_data.select("Metadata_JCP2022").unique().to_series().drop_nulls().to_list())

print("Querying broad-babel for perturbation types and names...")
# Fetch mapping for 'negcon' / 'trt'
pert_mapper = get_mapper(jcp_ids, input_column="JCP2022", output_columns="JCP2022,pert_type")
# Fetch mapping for readable drug names
name_mapper = get_mapper(jcp_ids, input_column="JCP2022", output_columns="JCP2022,standard_key")

# 3. Add the mapped columns to the dataset
# We prefix them with 'Metadata_' so they match the naming convention
plate_data = plate_data.with_columns(
    pl.col("Metadata_JCP2022").replace_strict(pert_mapper, default="unknown").alias("Metadata_pert_type"),
    pl.col("Metadata_JCP2022").replace_strict(name_mapper, default="unknown").alias("Metadata_name")
)

# 4. REORDER COLUMNS: Force all Metadata columns to the front, followed by features
meta_cols = [col for col in plate_data.columns if col.startswith("Metadata_")]
feat_cols = [col for col in plate_data.columns if not col.startswith("Metadata_")]
plate_data = plate_data.select(meta_cols + feat_cols)

# 5. Convert to Pandas and Save to CSV
df_plate = plate_data.to_pandas()
CSV_NAME = DATA_DIR / f"plate_{selected_plate}_exploration.csv"
df_plate.to_csv(CSV_NAME, index=False)

print(f"Successfully exported {selected_plate} to {CSV_NAME}")
print(f"Rows: {df_plate.shape[0]}, Columns: {df_plate.shape[1]}")
print(f"\nControl Types Found:\n{df_plate['Metadata_pert_type'].value_counts()}")

Processing data for 1053597806...
Querying broad-babel for perturbation types and names...
Successfully exported 1053597806 to ..\data\plate_1053597806_exploration.csv
Rows: 382, Columns: 3186

Control Types Found:
Metadata_pert_type
trt       318
poscon     32
negcon     32
Name: count, dtype: int64
